In [60]:
import os 
import json
import pandas as pd

In [73]:
models = ['linevul', 'codebert', 'PLBART', 'devign',  'regvd', 'vulberta_cnn', 'vulberta_mlp']

In [74]:
with open("../data/devign/test.jsonl", 'r') as f:
    data = []
    for  line in f:
        try:
            data.append(json.loads(line.strip()))
        except Exception as e:
            print(e)


In [75]:
scores = {model: {} for model in models}
for model in models:
    model_d = json.load(open(os.path.join(model, "line_scores.json"), "r"))
    for x in model_d:
        scores[model][x['idx']] = x['scores']

In [76]:
predictions = {model:{} for model in models}
for model in models:
    df_pred = pd.read_csv("predictions/{}_predictions.csv".format(model))
    for idx, row in df_pred.iterrows():
        predictions[model][row['idx']] = row['prediction']

In [77]:
def maximum_aggrement(freq, model_threshold):
    aggrement_lines = []
    for i, fr in enumerate(freq):
        if fr >= model_threshold: 
            aggrement_lines.append(i)
    return aggrement_lines

In [78]:
import shutil

In [83]:
model_threshold = 4
line_threshold = 20
minimum_line_threshold = 7
total_cnt = 0
for sample in data:
    if sample['target'] == 0: 
        continue

    ind = sample['idx']
    lines = sample['func'].split("\n")
    freq = [0] * len(lines)
    models_score_ids = {}
    
    fre_threshold = 0
    for model in models:
        if ind in predictions[model] and predictions[model][ind] == sample['target']:
            fre_threshold += 1
            paired_data_1 = [(scores[model][ind][i] if scores[model][ind][i] !=0 else -2 , i) for i in range(len(scores[model][ind]))]
            assert len(paired_data_1) == len(lines)
            sorted_data_1 = sorted(paired_data_1, reverse=True)
            models_score_ids[model] = [0] * len(lines)
            for id_, sorlin in enumerate(sorted_data_1):
                models_score_ids[model][sorlin[1]] = id_ if sorlin[0] !=-2 else -1

            data_1_lines = set([k[1] for k in sorted_data_1[:10]])
            for line_no in data_1_lines:
                freq[line_no] += 1

    if fre_threshold < model_threshold:
        continue

    lines_c = lines.copy()
    while lines_c.count('') > 0:
        lines_c.remove('')
    
    aggrement_lines = maximum_aggrement(freq, model_threshold)
    if len(aggrement_lines) >= minimum_line_threshold and len(lines_c) >= line_threshold:
        total_cnt += 1
        print("--------------------------------------------------------------------------------------------------------------")
        print("Example ids: ", sample['idx'], "project: ", sample['project'], "commit id: ", sample['commit_id'], "Number of lines: ", len(lines_c), )
        print("Original lines: ")
        print("Predicted Correctly by : ", [model for model in models_score_ids.keys()])
        max_length = max([len(line) for line in lines]) + 15

        for i in range(len(lines)):
            if len(lines[i]) == 0:
                continue
            current_length = len(str(i) + " ") +len(lines[i])
            print(i, lines[i], ' ' * (max_length - current_length), {model: models_score_ids[model][i] for model in models_score_ids.keys()})
            
        print("-----------------------")
        print("Highlighted lines: ")
        for ind  in aggrement_lines:
            print(ind, lines[ind])
print("--------------------------------------------------------------------------------------------------------------")
print(total_cnt)

--------------------------------------------------------------------------------------------------------------
Example ids:  265 project:  FFmpeg commit id:  478f1c3d5e5463a284ea7efecfc62d47ba3be11a Number of lines:  20
Original lines: 
Predicted Correctly by :  ['linevul', 'codebert', 'PLBART', 'devign', 'regvd', 'vulberta_cnn', 'vulberta_mlp']
0 static int decode_plte_chunk(AVCodecContext *avctx, PNGDecContext *s,               {'linevul': 2, 'codebert': 0, 'PLBART': 1, 'devign': 8, 'regvd': 7, 'vulberta_cnn': 11, 'vulberta_mlp': 13}
2                              uint32_t length)                                       {'linevul': 1, 'codebert': 9, 'PLBART': 2, 'devign': 10, 'regvd': 10, 'vulberta_cnn': 4, 'vulberta_mlp': 3}
4 {                                                                                   {'linevul': 19, 'codebert': 14, 'PLBART': 4, 'devign': -1, 'regvd': 17, 'vulberta_cnn': 8, 'vulberta_mlp': 17}
6     int n, i, r, g, b;                                           